In [2]:
import pandas as pd
import numpy as np
import geopy.distance

In [3]:
url = 'https://raw.githubusercontent.com/audreychu/Airbnb_Optimization/main/datasets/deduped_listings_updated.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude
0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000
1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344
2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052
3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018
4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356


In [5]:
# check that ids are all unique
df.id.nunique()

40135

In [7]:
# drop lat and long na
df = df[df['latitude'].notna()]
df = df[df['longitude'].notna()]
df = df.reset_index()

### 1. Subway Stations

In [9]:
import requests
from sodapy import Socrata

In [10]:
client = Socrata("data.cityofnewyork.us", 'Fwb56JZczDClDBHMEYOdmWugu')
results = client.get("kk4q-3rt2")
subway_df = pd.DataFrame.from_records(results)

In [28]:
df_coord = (df.longitude[20],df.latitude[20])
df_coord

(-73.99214, 40.71958)

In [26]:
# calculate number of subway lines
subway_df['line_count'] = subway_df.line.str.count("-")+1

In [36]:
# Count the number of stations within 0.2 miles
# count the number of subway lines within 0.2 miles
# This is technically double counting, but its okay
# since we are just trying to get a proxy for 
# transportation convenience

miles = 0.2

def sub_stations(df_coord,miles):
    cnt = 0
    line_cnt = 0
    for i in range(len(subway_df)):
        x = subway_df['the_geom'][i]['coordinates']
        if geopy.distance.geodesic(df_coord,x).miles < miles:
                cnt += 1
                line_cnt += subway_df.line_count[i]
    return(cnt,line_cnt)

#count_stations(df_coord,0.2)

In [37]:
sub_cols = []
miles = 0.2

for i in range(len(df)):
    df_coord = (df.longitude[i],df.latitude[i])
    sub_cols.append(sub_stations(df_coord,miles))
    
    

In [42]:
sub_cnt = list(zip(*sub_cols))[0]

In [43]:
sub_line_cnt = list(zip(*sub_cols))[1]

In [44]:
df.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000
1,1,1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344
2,2,2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052
3,3,3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018
4,4,4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356


In [45]:
df['sub_cnt_20m'] = sub_cnt

In [46]:
df['sub__line_cnt_20m'] = sub_line_cnt

In [47]:
df.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000,1,1
1,1,1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344,1,3
2,2,2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052,3,8
3,3,3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018,0,0
4,4,4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356,8,20


In [48]:
df.describe()

,level_0,index,Unnamed: 0,id,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m
count,40135.000000,40135.000000,40135.000000,4.013500e+04,40135.000000,40135.000000,40135.000000,40135.000000
mean,20067.000000,20067.000000,470224.111524,1.915594e+07,40.728933,-73.949870,1.940227,4.180491
std,11586.120864,11586.120864,163672.487669,1.141606e+07,0.055234,0.049202,1.958237,5.079378
min,0.000000,0.000000,13.000000,2.515000e+03,40.506410,-74.244420,0.000000,0.000000
25%,10033.500000,10033.500000,426192.000000,9.155944e+06,40.689050,-73.983850,1.000000,1.000000
50%,20067.000000,20067.000000,552538.000000,2.000018e+07,40.723650,-73.954390,1.000000,3.000000
75%,30100.500000,30100.500000,570515.500000,2.877463e+07,40.763470,-73.932475,3.000000,5.000000
max,40134.000000,40134.000000,591985.000000,4.037082e+07,40.911708,-73.712990,14.000000,32.000000


In [79]:
df.to_csv('df_subway_stations.csv')

### 2. Hotels

In [152]:
hot = client.get("tjus-cn27",limit=3000)
hotels = pd.DataFrame.from_records(hot)

In [162]:
hotels.describe()

,level_0,index
count,2724.000000,2724.000000
mean,1361.500000,1362.852423
std,786.495391,787.595990
min,0.000000,0.000000
25%,680.750000,681.750000
50%,1361.500000,1362.500000
75%,2042.250000,2043.250000
max,2723.000000,2730.000000


In [160]:
hotels = hotels[pd.notnull(hotels['latitude'])]
hotels = hotels[pd.notnull(hotels['longitude'])].reset_index()

In [161]:
miles = 0.2

def count_hotels(df_coord,miles):
    cnt = 0
    line_cnt = 0
    for i in range(len(hotels)):
        x = (hotels['longitude'][i],hotels['latitude'][i])
        if geopy.distance.geodesic(df_coord,x).miles < miles:
                cnt += 1
    return(cnt)

count_hotels(df_coord,0.2)

34

hotel_cols = []
miles = 0.2

for i in range(len(df)):
    df_coord = (df.longitude[i],df.latitude[i])
    hotel_cols.append(count_hotels(df_coord,miles))
    

In [163]:
df.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000,1,1
1,1,1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344,1,3
2,2,2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052,3,8
3,3,3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018,0,0
4,4,4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356,8,20


In [103]:
hotels_zip = hotels[['postcode','borough']].groupby(['postcode']).count().reset_index()
hotels_zip.columns =['postcode','count_hotel']
hotels_zip

,postcode,count_hotel
0,0,0
1,10001,66
2,10002,35
3,10003,39
4,10004,6
...,...,...
129,11434,14
130,11435,20
131,11436,6
132,11691,4


In [111]:
df_sub_hotel = df.merge(hotels_zip,left_on='zipcode',right_on='postcode',how='left')
df_sub_hotel.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m,postcode,count_hotel
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000,1,1,10025,49.0
1,1,1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344,1,3,11238,3.0
2,2,2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052,3,8,10001,66.0
3,3,3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018,0,0,11211,12.0
4,4,4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356,8,20,10013,419.0


In [112]:
df_sub_hotel.describe()

,level_0,index,Unnamed: 0,id,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m,count_hotel
count,40135.000000,40135.000000,40135.000000,4.013500e+04,40135.000000,40135.000000,40135.000000,40135.000000,33844.000000
mean,20067.000000,20067.000000,470224.111524,1.915594e+07,40.728933,-73.949870,1.940227,4.180491,37.788943
std,11586.120864,11586.120864,163672.487669,1.141606e+07,0.055234,0.049202,1.958237,5.079378,99.495395
min,0.000000,0.000000,13.000000,2.515000e+03,40.506410,-74.244420,0.000000,0.000000,0.000000
25%,10033.500000,10033.500000,426192.000000,9.155944e+06,40.689050,-73.983850,1.000000,1.000000,3.000000
50%,20067.000000,20067.000000,552538.000000,2.000018e+07,40.723650,-73.954390,1.000000,3.000000,6.000000
75%,30100.500000,30100.500000,570515.500000,2.877463e+07,40.763470,-73.932475,3.000000,5.000000,20.000000
max,40134.000000,40134.000000,591985.000000,4.037082e+07,40.911708,-73.712990,14.000000,32.000000,517.000000


In [114]:
df_sub_hotel.id.nunique()

40135

In [170]:
df_sub_hotel.to_csv('df_subway_stations_hotels.csv')

In [126]:
df_sub_hotel.count_hotel.isna().sum()

6291

In [168]:
miss_count = df_sub_hotel[(df_sub_hotel.count_hotel.isna()) | (df_sub_hotel['count_hotel']==0)]
len(miss_count)

6608

In [140]:
df_sub_hotel[(df_sub_hotel.count_hotel.isna()) | (df_sub_hotel['count_hotel']==0)].zipcode.value_counts()

10009         911
11385         317
11102         249
11105         199
11236         167
             ... 
10174.0         1
11433.0         1
11378.0         1
11385-2308      1
11417.0         1
Name: zipcode, Length: 210, dtype: int64

In [141]:
df_sub_hotel[(df_sub_hotel.count_hotel.isna()) | (df_sub_hotel['count_hotel']==0)].neighbourhood.value_counts()

Manhattan              1263
Queens                  918
Brooklyn                532
East Village            363
Alphabet City           243
                       ... 
Huguenot                  1
Pleasant Plains           1
Brighton Beach            1
Kingsbridge Heights       1
Eltingville               1
Name: neighbourhood, Length: 152, dtype: int64

In [169]:
df_sub_hotel.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude,sub_cnt_20m,sub__line_cnt_20m,postcode,count_hotel
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000,1,1,10025,49.0
1,1,1,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344,1,3,11238,3.0
2,2,2,132,27024.0,2019-03-07,Chelsea,Chelsea,Manhattan,New York,NY,10001,40.74968,-74.00052,3,8,10001,66.0
3,3,3,268,62264.0,2019-03-07,Williamsburg,Williamsburg,Brooklyn,Brooklyn,NY,11211,40.72159,-73.96018,0,0,11211,12.0
4,4,4,324,70381.0,2019-03-07,Soho,SoHo,Manhattan,New York,NY,10013,40.72195,-74.00356,8,20,10013,419.0


In [129]:
df_sub_hotel[df_sub_hotel['count_hotel']==0].neighbourhood.value_counts()

Queens       163
Ridgewood    126
Glendale      21
Brooklyn       4
Bushwick       2
Name: neighbourhood, dtype: int64

In [124]:
df_sub_hotel[df_sub_hotel['count_hotel']==0].city.value_counts()

Queens                   242
Ridgewood                 24
New York                  14
Ridgewood                 11
Brooklyn                   7
Glendale                   5
Glendale                   4
Brooklyn                   2
New york                   1
New York City              1
NY                         1
New york , Ridgewood       1
Brookyln                   1
Ridgewood Queens.          1
New York                   1
Bushwick                   1
Name: city, dtype: int64

In [150]:
hot_ls = hotels_zip.postcode.unique()
len(hot_ls)

134

In [151]:
df_ls = df.zipcode.unique()
len(df_ls)

342

In [149]:
len(list(set(df_ls) & set(hot_ls)))

132

### 1. Average household income

In [130]:
#inc = pd.read_csv('/Users/audreychu/Desktop/1001_Data/CCC Data Download_20201114_162426118/medianincomes.csv')
inc = pd.read_csv('/Users/audreychu/Desktop/1001_Data/inc_zip.csv')
#inc = inc[inc['Household Type'] == 'All Households']
inc.head()

,#,Zip Code,lat,long,City,Population,Avg. Income/H/hold,National Rank
0,1,10007,40.713973,-74.008336,"New York, New York","3,522","$112,947.00",#132
1,2,10280,40.708501,-74.016828,"New York, New York","6,614","$108,536.00",#157
2,3,10162,40.769298,-73.949942,"New York, New York","1,726","$108,416.00",#158
3,4,10004,40.691870,-74.014109,"New York, New York","1,225","$101,868.00",#238
4,5,10069,40.775551,-73.989689,"New York, New York","1,403","$88,091.00",#507


In [165]:
avg_inc = []
for j in range(10000):
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    avg_inc.append(inc['Avg. Income/H/hold'][ind])

In [167]:
avg_inc_10_40 = []

for j in range(10001,40000):
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    avg_inc_10_40.append(inc['Avg. Income/H/hold'][ind])

In [169]:
avg_inc_40_70 = []

for j in range(40001,70000):
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    avg_inc_40_70.append(inc['Avg. Income/H/hold'][ind])

In [170]:
avg_inc_70_100 = []

for j in range(70001,100000):
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    avg_inc_70_100.append(inc['Avg. Income/H/hold'][ind])

In [195]:
avg_inc_100_ = []

for j in range(100001,len(df)):
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    avg_inc_100_.append(inc['Avg. Income/H/hold'][ind])

In [196]:
#10000
#40000
#70000
#100000

fix = [10000,40000,70000,100000]
fix_ls = []

for j in fix:
    df_coord = (df.latitude[j], df.longitude[j])
    dist = []
    for i in range(len(inc)):
        inc_coord = (inc.lat[i], inc.long[i])
        dist.append(geopy.distance.geodesic(df_coord,inc_coord).miles)
    ind = dist.index(min(dist))
    fix_ls.append(inc['Avg. Income/H/hold'][ind])

In [197]:
fix_ls

['$40,176.00 ', '$49,976.00 ', '$40,176.00 ', '$17,970.00 ']

In [200]:
len(df)

114129

avg_inc.append(fix_ls[0])

avg_inc_10_40.append(fix_ls[1])
avg_inc_40_70.append(fix_ls[2])
avg_inc_70_100.append(fix_ls[3])

In [227]:
avg_sal = avg_inc + avg_inc_10_40 + avg_inc_40_70 + avg_inc_70_100 + avg_inc_100_

In [228]:
df['avg_salary'] = avg_sal

In [229]:
df.head()

,level_0,index,Unnamed: 0,id,last_scraped,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,latitude,longitude,avg_salary
0,0,0,13,5435.0,2019-03-07,Manhattan,Upper West Side,Manhattan,New York,NY,10025,40.80155,-73.97000,"$49,733.00"
1,1,1,20,7036.0,2019-03-07,Kensington,Kensington,Brooklyn,Brooklyn,NY,11218,40.64522,-73.98293,"$101,868.00"
2,2,2,77,17298.0,2019-03-07,Bedford-Stuyvesant,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11233,40.68291,-73.93664,"$24,022.00"
3,3,3,78,17299.0,2019-03-07,Brooklyn,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11233,40.68276,-73.93503,"$24,022.00"
4,4,4,96,20311.0,2019-03-07,Brooklyn,Prospect Heights,Brooklyn,Brooklyn,NY,11238,40.67206,-73.96344,"$101,868.00"


In [231]:
df.to_csv('df_salary.csv')

import geopandas as gpd
import pandas as pd
import pyproj

df1 = pd.read_csv("/home/ubuntu/maid80.csv")
df2 = pd.read_csv("/home/ubuntu/iodr.csv")

gdf1 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['longitude'], df1['latitude']), crs=pyproj.CRS.from_epsg(4326))
gdf2 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['longitude'], df2['latitude']), crs=pyproj.CRS.from_epsg(4326))

radius = 400
for gdf in [gdf1, gdf2]:
  gdf.to_crs(pyproj.CRS.from_epsg(3857), inplace=True)

gdf1['geometry'] = gdf1['geometry'].buffer(radius)
gdf2['IS_WITHIN_400M'] = 1

gdf = gpd.sjoin(gdf1, gdf2['geometry'], how='left')
print(gdf[gdf.IS_WITHIN_400M_right==1].head())

### Distance from subway entrance

In [233]:
sub['long'] = [x.split('(')[1] for x in sub['longitude']]
sub['lat'] = [x.split(')')[0] for x in sub['latitude']]
sub = sub.reset_index()

In [234]:
df_test = (df.latitude[0],df.longitude[0])
sub_test = (sub.lat[0],sub.long[0])

geopy.distance.geodesic(df_test,sub_test).miles

6.259646418494054

In [235]:
x = geopy.distance.geodesic(df_test,sub_test).miles
x < 0.5

False

#### Calculate number of subway entrances within x miles

In [236]:
def count_entrances(df_test,miles):
    cnt = 0
    for i in range(len(sub)):
        x = (sub.lat[i],sub.long[i])
        if geopy.distance.geodesic(df_test,x).miles < miles:
                cnt += 1
    return(cnt)

In [237]:
df_coord=(df.latitude[1],df.longitude[1])
cnt = count_entrances(df_coord,0.25)
cnt

4

In [243]:
num_sub = []
for i in range(10000):
    df_coord = (df.latitude[i],df.longitude[i])
    x = count_entrances(df_coord,0.20)
    num_sub.append(x)

In [242]:
len(df)

114129

In [246]:
num_sub[:100]

[5,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 14,
 0,
 10,
 2,
 6,
 0,
 4,
 2,
 4,
 2,
 0,
 8,
 3,
 0,
 0,
 4,
 0,
 13,
 4,
 0,
 27,
 4,
 4,
 2,
 4,
 4,
 5,
 0,
 0,
 4,
 3,
 6,
 4,
 5,
 1,
 2,
 6,
 0,
 0,
 0,
 4,
 6,
 4,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 4,
 5,
 4,
 0,
 5,
 3,
 5,
 6,
 18,
 17,
 10,
 2,
 17,
 10,
 2,
 4,
 0,
 0,
 8,
 9,
 0,
 0,
 0,
 17,
 2,
 5,
 0,
 3,
 0,
 4,
 3,
 0,
 0,
 0,
 0,
 24,
 3,
 4]

In [52]:
df.iloc[4190]

Unnamed: 0                            36281
id                              2.52167e+07
last_scraped                     2019-03-06
neighbourhood                           NaN
neighbourhood_cleansed                  NaN
neighbourhood_group_cleansed            NaN
city                                    NaN
state                                   NaN
zipcode                                 NaN
latitude                                NaN
longitude                               NaN
Name: 4190, dtype: object